In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
!nvidia-smi

In [ ]:
from PIL import Image
Image.open("train/0/0.png")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
hand = pd.read_csv("hand_gesture_pose.csv")
hand

In [ ]:
import glob
train = pd.DataFrame({"path":glob.glob("train/*")})
train["path"] = sorted(train["path"].apply(lambda x:int(x.split("\\")[1])))
train["path"] = train["path"].astype("str") 
train["path"] = "train/" + train["path"] + "/0.png"
train

In [ ]:
json_df = pd.DataFrame({"path":glob.glob("train/*/*")})
json_df = json_df[json_df['path'].str.contains("json")]
json_df["path"] = sorted(json_df["path"].apply(lambda x:int(x.split("\\")[1])))
json_df["path"] = json_df["path"].astype("str")
json_df["path"] = "train/" + json_df["path"] + "/" + json_df["path"] + ".json"
json_df

In [ ]:
import json
answers = []
for i in json_df["path"]:
    with open(i) as json_file:
        json_data = json.load(json_file)
        cat = json_data.get('action')[0]
        cat_name = json_data.get('action')[1]
        answers.append([cat, cat_name])
len(answers)

In [ ]:
answers = pd.DataFrame(answers)
answers

In [ ]:
df = pd.concat([train,answers],axis = 1)
df.columns = ["path","answer","answer_name"]
df["answer"] = df["answer"].astype("str")
df["path"] = df["path"].astype("str")
df["folder"] = df["path"].apply(lambda x:x.split("/")[1])
df

In [ ]:
new_train_df = pd.DataFrame({"path":glob.glob("train/*/*.png")})
new_train_df["folder"] = new_train_df["path"].apply(lambda x:x.split("\\")[1])
new_train_df = new_train_df.merge(df[["answer","folder"]], on = "folder", how = "left")
new_train_df

In [ ]:
test = pd.DataFrame({"path":glob.glob("test/*/*.png")})
test["folder"] = test["path"].apply(lambda x:x.split("\\")[1])
test["folder"].nunique()

In [ ]:
test

In [ ]:
test.shape

In [ ]:
from keras import *
from keras.layers import *
from keras.callbacks import *
from keras.applications.efficientnet import EfficientNetB3
#from keras.optimizers import adam_v2 
import warnings
warnings.filterwarnings(action="ignore")
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 82)
result = 0

for i, (train_index, valid_index) in enumerate(skf.split(new_train_df,new_train_df["answer"])):
    X_train = new_train_df.iloc[train_index]
    X_valid = new_train_df.iloc[valid_index]
    idg = ImageDataGenerator() # zoom_range = 0.4
    idg2 = ImageDataGenerator()
    idg3 = ImageDataGenerator()
    
    train_generator = idg.flow_from_dataframe(X_train,x_col = "path", y_col = "answer", batch_size = 16, target_size=(280,280))   
    valid_generator = idg2.flow_from_dataframe(X_valid,x_col = "path", y_col = "answer", batch_size = 16, target_size=(280,280))
    test_generator = idg3.flow_from_dataframe(test, x_col = "path", y_col = None, target_size = (280,280), batch_size = 16, class_mode = None, shuffle = False)
    
    es = EarlyStopping(patience = 5, verbose = 1,restore_best_weights = True)
    #mc = ModelCheckpoint(f"best{i}.h5",save_best_only = True, verbose = 1) 
    rl = ReduceLROnPlateau(patience = 3, verbose = 1,factor = 0.25)
    model = Sequential()
    efficient = EfficientNetB3(include_top = False, weights = "imagenet", pooling = "avg")
    model.add(efficient)
    model.add(Dense(157,activation = "softmax"))
    model.compile(metrics = ["acc"], optimizer = "adam", loss = "categorical_crossentropy")
    model.fit(train_generator, validation_data = valid_generator, epochs = 100, callbacks = [es,rl],verbose = 1) #, steps_per_epoch=1, validation_steps = 1
    
    result += model.predict(test_generator, verbose = 1) / 5
    

In [ ]:
train_index = list(train_generator.class_indices)

In [ ]:
sub_path = ["Image_Path"]  +  ["Label_" + x for x in train_index] 

In [ ]:
result_df = pd.DataFrame(result)
result_df

In [ ]:
result_df["folder"] = test["folder"]

In [ ]:
test.shape, result.shape

In [ ]:
result_df["folder"].nunique()

In [ ]:
result_df_array = np.array(result_df.groupby("folder").mean())
result_df_array.shape

In [ ]:
result.shape

In [ ]:
pd.options.display.max_columns = 9999
sub = pd.read_csv("sample_submission.csv")
sub

In [ ]:
a = [["Image_Path","Label_0","Label_1","Label_2",'Label_3',"Label_4","Label_5","Label_6","Label_7","Label_8","Label_9","Label_10","Label_11","Label_12","Label_13","Label_14","Label_15","Label_16","Label_17","Label_18","Label_19","Label_22","Label_23","Label_25","Label_26","Label_27","Label_28","Label_29","Label_31","Label_32","Label_34","Label_35","Label_36","Label_37","Label_39","Label_40","Label_41","Label_42","Label_43","Label_44","Label_47","Label_48","Label_49","Label_50","Label_51","Label_53","Label_54","Label_55","Label_56","Label_57","Label_58","Label_59","Label_60","Label_61","Label_62","Label_64","Label_65","Label_66","Label_67","Label_68","Label_69","Label_70","Label_72","Label_73","Label_74","Label_75","Label_77","Label_78","Label_79","Label_80","Label_81","Label_82","Label_83","Label_85","Label_87","Label_88","Label_89","Label_90","Label_92","Label_93","Label_94","Label_95","Label_97","Label_98","Label_100","Label_101","Label_105","Label_106","Label_107","Label_109","Label_110","Label_111","Label_112","Label_113","Label_114","Label_115","Label_116","Label_117","Label_118","Label_119","Label_122","Label_123","Label_124","Label_125","Label_126","Label_128","Label_129","Label_131","Label_132","Label_133","Label_134","Label_135","Label_136","Label_137","Label_138","Label_139","Label_140","Label_141","Label_142","Label_143","Label_144","Label_145","Label_146","Label_147","Label_148","Label_149","Label_150","Label_151","Label_153","Label_154","Label_155","Label_156","Label_157","Label_158","Label_159","Label_160","Label_161","Label_162","Label_163","Label_165","Label_167","Label_168","Label_169","Label_171","Label_172","Label_173","Label_174","Label_175","Label_177","Label_186","Label_188","Label_189","Label_190","Label_191","Label_192","Label_193","Label_194","Label_195"]]
a = sub.columns
a

In [ ]:
sub.iloc[:,1:] = result_df
sub.columns = sub_path
sub = sub[a]
sub

In [ ]:
sub.to_csv("sub17.csv",index=False)